## Prepairing functions for alignments

In [2]:
import re
import numpy as np
import itertools
import pandas
import scipy as scp

In [3]:
# Functions to work with strings and profiles

def str_to_profile(string):
    return [{col : 1} for col in string]

def profile_to_str(profile):
    return ''.join(tuple(col.keys())[0] for col in profile)

# Pairwise alignment

def align_pairwise(a, b, M, score_needed=False): # a и b - profiles, M - weight matrix
    
    # !!!!Вставим функции сюда, чтобы не волноваться о пространстве имён!!!
    
    # Weight for alignment
    def weight(a, b, M):
        var_1, var_2 = 0, 0
        for s_a in a:
            for s_b in b:
                var_1 += a[s_a]*b[s_b] * M[amino_ind[s_a], amino_ind[s_b]]
                var_2 += a[s_a]*b[s_b]
        return var_1/var_2

    # Merging two profiles
    def merge(a, b):
        merge = a.copy()
        for key in b:
            merge[key] = merge.get(key, 0) + b[key]
        return merge

    # Fulfiling borders
    def gap_w(s0, s1_w):
        w = np.zeros((len(s0)+1))
        for i in range(1, len(s0)+1):
            w[i] = w[i-1] + weight(s0[i-1], {'-': s1_w}, M)
        return w
    
    a_w, b_w = sum(a[0].values()), sum(b[0].values())
    
    # Making our alignment-table
    
    n, m = len(a), len(b)
    D = np.empty((n+1, m+1))
    D[:,0] = gap_w(a, b_w)
    D[0,:] = gap_w(b, a_w)
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            a_gap = D[i, j-1] + weight(b[j-1], {'-': a_w}, M)
            b_gap = D[i-1, j] + weight(a[i-1], {'-': b_w}, M)
            match = D[i-1, j-1] + weight(a[i-1], b[j-1], M)
            D[i, j] = max(a_gap, b_gap, match)

    # Making consensus profile
    i, j = n, m
    consensus = []
    
    if score_needed:
        # For printifs
        return D[-1,-1] 

    while i + j > 0:
        if D[i, j] == D[i-1, j] + weight(a[i-1], {'-': b_w}, M):
            consensus.append(merge(a[i-1], {'-': b_w}))
            i -= 1
        elif D[i, j] == D[i, j-1] + weight(b[j-1], {'-': a_w}, M):
            consensus.append(merge(b[j-1], {'-': a_w}))
            j -= 1
        else:
            consensus.append(merge(b[j-1], a[i-1]))
            i -= 1
            j -= 1
    return consensus[::-1]

In [4]:
# Weighted Pair Group Method with arithmetic mean

def WPGMA(ind_in, A_in):
    
    # Finding min elem
    argmin = lambda arr: np.unravel_index(arr.argmin(), arr.shape)
    
    # Converting A_in
    ind = ind_in.copy()
    A = A_in.copy()
    A -= np.tri(*A.shape)
    A = A.flatten()
    A[A < 0] = np.inf
    A = A.reshape((len(ind), len(ind)))
    A[-1,:] = 0

    # Constructing tree
    while len(ind)-1:
        i, j = argmin(A[:-1,:])
        w = A[i,j]
        
        # dealing with i and i,j
        ind[i] = '(' + ind[i] + ':' + str(w/2 - A[-1,i]) + ', ' + ind[j] + ':' + str(w/2 - A[-1,j]) + ')'
        del ind[j]
        A[:-1,i] = (A[:-1,i] + A[:-1,j])/2
        A[-1,i] += w/2 - A[-1,i]
        
        # deleting i-string and j-column
        
        if j == A.shape[0] - 1:
            A = np.hstack((A[:,:j],A[:,j+1:]))
        else:
            A = np.vstack((np.hstack((A[:j,:j],A[:j,j+1:])),np.hstack((A[j+1:,:j],A[j+1:,j+1:]))))
        
    return ind[0]

In [5]:
# Generating table for creating tree

def mutation_dist(string_mass, W):
    
    n = len(string_mass)
    ind = list(string_mass.keys())
    M = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
            M[i,j] = -align_pairwise(string_mass[ind[i]], string_mass[ind[j]], W, score_needed=True)
            
    M[M < 0] -= (M.min() - 1)
    
    return ind, M

In [6]:
# Creating consensus for tree

def multiple_alignment(strings_mass, tree, W):
    
    tr = tree
    i = len(strings_mass)
    strings_tab = strings_mass.copy()
    
    while re.search(r'\((s_\d+)[^\)^\(]+?(s_\d+).+?\)', tr):
        
        dns = re.findall(r'\((s_\d+)[^\)^\(]+?(s_\d+).+?\)', tr)
        
        for node in dns:
            tr = re.sub(r'\({}[^\)^\(]+?{}.+?\)'.format(*node), 's_' + str(i), tr)
            strings_tab['s_'+str(i)] = align_pairwise(strings_tab[node[0]], strings_tab[node[1]], W)
            i += 1

    return strings_tab[tr]

In [16]:
# Alignment string profile with consensus

def align_pair_cons(cons, s, M): # cons and s - profiles, M - weight matrix
    
    cons_w, s_w = sum(cons[0].values()), sum(s[0].values())
    
    # Weight for alignment
    def weight(a, b, M):
        var_1, var_2 = 0, 0
        for s_a in a:
            for s_b in b:
                var_1 += a[s_a]*b[s_b] * M[amino_ind[s_a], amino_ind[s_b]]
                var_2 += a[s_a]*b[s_b]
        return var_1/var_2

    # Merging two profiles
    def merge(a, b):
        merge = a.copy()
        for key in b:
            merge[key] = merge.get(key, 0) + b[key]
        return merge

    # Fulfiling borders
    def gap_w(s0, s1_w):
        w = np.zeros((len(s0)+1))
        for i in range(1, len(s0)+1):
            w[i] = w[i-1] + weight(s0[i-1], {'-': s1_w}, M)
        return w
    
    # Fulfiling table
    n, m = len(cons), len(s)
    D = np.empty((n+1, m+1))
    D[:,0] = gap_w(cons, s_w)
    D[0,:] = gap_w(s, cons_w)
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            s_gap = D[i-1, j] + weight(cons[i-1], {'-': s_w}, M)
            match = D[i-1, j-1] + weight(cons[i-1], s[j-1], M)
            D[i, j] = max(s_gap, match)
            
    # Making alignments
    
    i, j = n, m
    s_to_cons = []

    while i + j > 0:
        if D[i, j] == D[i-1, j] + weight(cons[i-1], {'-': s_w}, M):
            s_to_cons.append({'-': 1})
            i -= 1
        else:
            s_to_cons.append(s[j-1])
            i -= 1
            j -= 1
                
    return profile_to_str(s_to_cons[::-1]).replace('-','_')

In [8]:
# Reading PAM_250

PAM_250 = np.empty((25,25), dtype=np.int)
ind = {}

with open('PAM_250.txt', 'r') as f:
    amino = f.readline().split()[1:]
    amino_ind = {amino[i]:i for i in range(25)}
    for i in range(25):
        PAM_250[i,:] = np.array(f.readline().split()[2:])

In [9]:
# Another attempts is to take default match = 1, mismatch = -1, gap = -1

PAM_default = np.diag(np.ones(25, dtype=np.int)*2) - np.ones((25,25), dtype=np.int)
PAM_default[-1,-1] = 0

Создаем независимо мутировавшие аминокислотные последовательности

In [10]:
# Add random mutations in our string

def mutations(string, num_insertion, num_deletion, max_len):
    for _ in range(num_insertion):
        pos= np.random.randint(0, len(string)-1)
        insertion = np.random.randint(0, max_len)
        string = string[:pos] + ''.join([amino[np.random.randint(0, 19)] for _ in range(insertion)]) + string[pos:]
    for _ in range(num_deletion):
        pos = np.random.randint(0, len(string)-1)
        dlt = np.random.randint(0, max_len)
        string = string[:pos] + string[pos+dlt:]
    return string

In [11]:
# Creating massive of peptids

def mutation_peptids(alphabet, max_ind):
    mother_seq = ''.join([alphabet[np.random.randint(0, max_ind)] for _ in range(40)])
    peptids = {}
    for i in range(20):
        peptids['s_' + str(i)] = mutations(mother_seq, np.random.randint(1, 5), np.random.randint(1, 5), 4)

    for s in peptids:
        print(peptids[s])
        peptids[s] = str_to_profile(peptids[s])

    return peptids

## Making alignments

In [12]:
strings_mass = mutation_peptids(amino, 10) # 10 - because i have wooden pc

ADQRRNGGQKCNICGGERIDIDHRGAQNCDRIWPCIDD
CHQGEAEDQRRNGGCNCGGERDIDHNLYRGAGQNCDIDD
CHQNGEAEDQRIWWRNGGCICGGERIDIDHICRGANCDRCIDD
CHQGEAEDQRRNGGCNICGNPGERIDIRGAGQNCDRDD
CHQGEDNQRRNGGCNICGGIFERIDIDHRGSAGQNCDRCIDD
CHQGEAEDQRRNGGCNICGGERIDIDHGKMYSMQNCDRCIDD
CHQGDQRRNFREGGCNICGGERIDIDHRGAGQNCDRCIDD
CHQGEAEAIIDQRRMGNGGCEWANIDIDHRGAGQNCDRCIDD
HQGEAEDSMMQRRGWCNICGGERILCFDIDHRGAGQNCCIDD
CHQGEAEDQNGGCNICGIERIDIDHRGAGQNCDRCIDD
HKCHQGEAEDQRRNGGLCNICGGERTIDHRGAGQNCDRCIDD
CKIHQGEAEDQRRNHGGCNTHYICGGERIIDHRGAGQNCDRCIDD
CHQGEADQRRNGGCNICGGERIDIDHGQNCIDD
LFCHQGEAEDQRRNGGERIDIDHRGAGQNCDRCIDD
CHQNKCHGEAEDQRRNMPGGCNICGGERIDIDHRGAGQNCDRCIDD
CHQGEAEDQRRYHCNICGGERIDIDHRGAGQNCDRCIDD
CHQGEAERRNGGCNIGTCGGERIDHRQNCDRCFSWIDD
CHQGEAEDQRRCNGGCNICGQSRIDIDHRGAGQNCDRCIDD
CHAEHQEDQRRNGGCNGLWDGERIDIDHRGAGQNCDRCIDD
HQGEAQRRNGGCNICGGERIDIDPNHRGAGQNNCIDD


In [17]:
# Multiple alignment using Pam_def

# Creating tree
tree = WPGMA(*mutation_dist(strings_mass, PAM_default))

# Creating consensus
cons = multiple_alignment(strings_mass, tree, PAM_default)

for key in strings_mass:
    print(align_pair_cons(cons, strings_mass[key], PAM_default))

_____________A_____D___QR___R_N___GGQKCN___I__CG__G__ER__I___DID__H___RG_A___QN_CDRIWPC___IDD
__C_____HQ_GEA_E___D___QR___R_N___GG__CN______CG__G__ER______DID__HNLYRG_AG__QN_CD________IDD
__C_____HQNGEA_E___D___QRIWWR_N___GG__C____I__CG__G__ER__I___DID__HIC_RG_A____N_CDR___C___IDD
__C_____HQ_GEA_E___D___QR___R_N___GG__CN___I__CGNPG__ER__I___DI_______RG_AG__QN_CDR________DD
__C_____HQ_GE______DN__QR___R_N___GG__CN___I__CG__GIFER__I___DID__H___RGSAG__QN_CDR___C___IDD
__C_____HQ_GEA_E___D___QR___R_N___GG__CN___I__CG__G__ER__I___DID__H____GKMYSMQN_CDR___C___IDD
__C_____HQ_G_______D___QR___R_NFREGG__CN___I__CG__G__ER__I___DID__H___RG_AG__QN_CDR___C___IDD
__C_____HQ_GEA_EAIID___QR___R_M___G____N_______G__GC_EWANI___DID__H___RG_AG__QN_CDR___C___IDD
________HQ_GEA_E___DSMMQR___R_____GW__CN___I__CG__G__ER__ILCFDID__H___RG_AG__QN_C_____C___IDD
__C_____HQ_GEA_E___D___Q______N___GG__CN___I__CG__I__ER__I___DID__H___RG_AG__QN_CDR___C___IDD
HKC_____HQ_GEA_E___D___QR___R_N___GGL_CN___I__CG__G__ER__T__

In [18]:
# Выравниваем по матрице PAM_250

# Creating tree
tree = WPGMA(*mutation_dist(strings_mass, PAM_250))

# Creating consensus
cons = multiple_alignment(strings_mass, tree, PAM_250)

for key in strings_mass:
    print(align_pair_cons(cons, strings_mass[key], PAM_250))

__A__D__Q_RRNG___GQ______K_____CN____I__CGG__ER_I___DIDH__RG__AQNCDRIWPCIDD
__C__H__Q_GEAE___DQ__R___RN_GG_CN_______CGG__ERDI___DHNL_YRG_AGQNCD_____IDD
__C__H__QNGEAE___DQ__RIWWRN_GG_C_____I__CGG__ER_I___DIDH__IC_RGANCDR___CIDD
__C__H__Q_GEAE___DQ__R___RN_GG_CN____I__CGN__PG_E___RIDI__RG_AGQNCDR_____DD
__C__H__Q_GE_D___NQ__R___RN_GG_CN____I__CGGIFER_I___DIDH__RGSAGQNCDR___CIDD
__C__H__Q_GEAE___DQ__R___RN_GG_CN____I__CGG__ER_I___DIDHG_KMYSMQNCDR___CIDD
__C__H__Q_GDQR___RN__F___RE_GG_CN____I__CGG__ER_I___DIDH__RG_AGQNCDR___CIDD
__C__H__Q_GEAEAIIDQ__R___RM_GN_GG_______CEW__AN_I___DIDH__RG_AGQNCDR___CIDD
_____H__Q_GEAE___DSMMQ___RR_GW_CN____I__CGG__ER_ILCFDIDH__RG_AGQNC_____CIDD
__C__H__Q_GEAE___DQ______N__GG_CN____I__CGI__ER_I___DIDH__RG_AGQNCDR___CIDD
K__C__H__Q_GEAE___DQ__R___RN_GGLCN____I__CGG__ER_T____IDH__RG_AGQNCDR___CIDD
__CKIH__Q_GEAE___DQ__R___RNHGG_CNTHY_I__CGG__ER_I____IDH__RG_AGQNCDR___CIDD
__C__H__Q_GEAD____Q__R___RN_GG_CN____I__CGG__ER_I___DIDH______GQNC______IDD
LFC__H__Q_G

#### It can be seen that PAM_250 can strongly change result